In [1]:
import sys; sys.argv=['']; del sys
import numpy as np
import os
from args import get_parser
from utils.dataloader import DataLoader
from utils.config import get_opt
from utils.lang_proc import idx2word, sample
from model import get_model
import pickle
import json
import time

Using TensorFlow backend.


In [2]:
parser = get_parser()
args_dict = parser.parse_args()
args_dict.mode = 'test'
args_dict.bs = 1
args_dict.cnn_train = True
args_dict.sgate = True
args_dict.model_file = '/work/asalvador/sat_keras/models/sgate_cnn_train_weights_.e8_CIDEr0.86.h5'

In [3]:
model = get_model(args_dict)
opt = get_opt(args_dict)

weights = args_dict.model_file
model.load_weights(weights)
vocab_file = os.path.join(args_dict.data_folder,'data',args_dict.vfile)
vocab = pickle.load(open(vocab_file,'rb'))
inv_vocab = {v:k for k,v in vocab.items()}

model.compile(optimizer=opt,loss='categorical_crossentropy')

/imatge/asalvador/workspace/sat_keras/venv/local/lib/python2.7/site-packages/keras/engine/topology.py:379: UserWarning: The `regularizers` property of layers/models is deprecated. Regularization losses are now managed via the `losses` layer/model property.
  warnings.warn('The `regularizers` property of layers/models '
/imatge/asalvador/workspace/sat_keras/venv/local/lib/python2.7/site-packages/keras/regularizers.py:18: UserWarning: The `set_param` method on regularizers is deprecated. It no longer does anything, and it will be removed after 06/2017.
  warnings.warn('The `set_param` method on regularizers is deprecated. '
/imatge/asalvador/workspace/sat_keras/venv/local/lib/python2.7/site-packages/keras/engine/topology.py:371: UserWarning: The `regularizers` property of layers/models is deprecated. Regularization losses are now managed via the `losses` layer/model property.
  warnings.warn('The `regularizers` property of '


In [4]:
dataloader = DataLoader(args_dict)
N_train, N_val, N_test = dataloader.get_dataset_size()
N = args_dict.bs
gen = dataloader.generator('test',batch_size=args_dict.bs,train_flag=False)

In [8]:
captions = []
num_samples = 0
print_every = 100
t = time.time()
k = 1
for [ims,prevs],caps,imids in gen:
    
    # beam search
    word_idxs = np.zeros((k,args_dict.seqlen))
    word_idxs[:,:] = 2
    ### beam search caps ###
    seqs,scores = beamsearch(model=model,image=ims,
                             vocab_size = args_dict.vocab_size, 
                             start=1,eos=2,maxsample=args_dict.seqlen,k=k)
    
    seqs = np.array(seqs)[np.argsort(scores)[::-1][:k]]
    for i,seq in enumerate(seqs):
        word_idxs[i,:len(seq)-1] = seq[1:]
    pred_caps = idx2word(word_idxs,inv_vocab)
    true_caps = idx2word(np.argmax(caps,axis=-1),inv_vocab)

    # true caption
    print "ID:", imids[0]['file_name']
    true_cap = ' '.join(true_caps[0])
    print "T:", true_cap

    print "B:",' '.join(pred_caps[0][:-1])
    model.reset_states()

    # greedy caps
    prevs = np.ones((N,1))
    word_idxs = np.zeros((N,args_dict.seqlen))

    for i in range(args_dict.seqlen):
        # get predictions
        preds = model.predict_on_batch([ims,prevs]) #(N,1,vocab_size)
        preds = preds.squeeze()

        word_idxs[:,i] = np.argmax(preds,axis=-1)
        prevs = np.argmax(preds,axis=-1)
        prevs = np.reshape(prevs,(N,1))

    pred_caps = idx2word(word_idxs,inv_vocab)
    #true_caps = idx2word(np.argmax(caps,axis=-1),inv_vocab)

    pred_cap = ' '.join(pred_caps[0][:-1])# exclude eos
    print "G:", pred_cap
    print "="*10
    
    model.reset_states()


ID: COCO_val2014_000000550529.jpg
T: a motorbike sitting in front of a wine display case <eos>
B: a black and white motorcycle is sitting on a wooden bench .
G: a black and white motorcycle is parked in a row .
ID: COCO_val2014_000000348881.jpg
T: a large jetliner sitting on top of an airport runway . <eos>
B: a man standing next to a white plane .
G: a man standing next to a white plane .
ID: COCO_val2014_000000560623.jpg
T: an airport filled with planes sitting on <unk> . <eos>
B: a view of a large airplane on the runway .
G: a large group of people standing in front of a plane .
ID: COCO_val2014_000000561100.jpg
T: a small blue plane sitting on top of a field . <eos>
B: a small airplane is flying in the air .
G: a small airplane is flying in the air .
ID: COCO_val2014_000000354533.jpg
T: a motorcycle sits parked across from a herd of livestock . <eos>
B: a man riding a motorcycle with a flag on it .
G: a man riding a motorcycle with a flag on it .
ID: COCO_val2014_000000334321.jpg
T

KeyboardInterrupt: 